In [ ]:
import os
import cv2
import glob
import shap
import numpy as np

from time import time
from sklearn.utils import shuffle
from matplotlib import pyplot as plt
from itertools import combinations, permutations

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.callbacks import LearningRateScheduler

os.environ["CUDA_VISIBLE_DEVICES"] = "1"

In [ ]:
# data parameters
num_classes = 10

In [ ]:
# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

# scale images to the [0, 1] range
x_train = x_train.astype("float32") / 255
x_test = x_test.astype("float32") / 255

x_train = np.expand_dims(x_train, -1)
x_test = np.expand_dims(x_test, -1)

print("x_train shape :", x_train.shape)
print("x_test shape :", x_test.shape)

In [ ]:
# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

print("y_train shape :", y_train.shape)
print("y_test shape :", y_test.shape)

In [ ]:
def lr_decay(epoch):
    init_lr = 0.01
    drop_rate = 0.85
    drop_epochs = 5
    
    # learning rate decay
    lr = init_lr * np.power(drop_rate, np.floor((1+epoch)/drop_epochs))
    
    if (1 + epoch) % drop_epochs == 0: 
        print('learning rate is decayed to %f' % lr)
        
    return lr

In [ ]:
# model
inputs = keras.Input(shape=(28, 28, 1))

x = layers.Conv2D(32, (3, 3), activation="relu", padding="same")(inputs)
x = layers.MaxPooling2D(pool_size=(2, 2))(x)
x = layers.Dropout(0.5)(x, training=True)

x = layers.Conv2D(64, (3, 3), activation="relu", padding="same")(x)
x = layers.MaxPooling2D(pool_size=(2, 2))(x)
x = layers.Dropout(0.5)(x, training=True)

x = layers.Flatten()(x)
outputs = layers.Dense(num_classes, activation="softmax")(x)

model = keras.Model(inputs=inputs, outputs=outputs)
model.summary()
model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

In [ ]:
batch_size = 512
lr_scheduler = LearningRateScheduler(lr_decay)
history = model.fit(x_train, y_train, batch_size=batch_size, epochs=100, validation_split=0.1, callbacks=[lr_scheduler], shuffle=True)

In [ ]:
plt.plot(history.history["loss"])
plt.plot(history.history["val_loss"])
plt.title("Loss")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.legend(["Train", "Val"])
plt.show()

plt.plot(history.history["accuracy"])
plt.plot(history.history["val_accuracy"])
plt.title("Accuracy")
plt.xlabel("Accuracy")
plt.ylabel("Loss")
plt.legend(["Train", "Val"])
plt.show()

In [ ]:
score = model.evaluate(x_test, y_test, verbose=0)

print("Test loss :", score[0])
print("Test accuracy :", score[1])

### SHAP for Target

In [ ]:
repeat = 30000
width = x_train.shape[1]
height = x_train.shape[2]
feat_dim = width * height

data = x_test[:2048]
importance = np.zeros(data.shape)
count = np.zeros(data.shape)

level = 1
level_set = np.arange(level)
level_set = np.concatenate([level_set, feat_dim-level_set-1])
level_set_prob = [0.5] * 2

In [ ]:
# determine m
start_time = time()

for i in range(0, repeat):
    # get level
    level = np.random.choice(level_set, 1, p=level_set_prob)[0]
    
    # 1-d mask
    mask = np.concatenate([np.ones(level), np.zeros(feat_dim-level)])
    np.random.shuffle(mask)
    
    # reshape mask to 2-d
    mask = np.reshape(mask, data.shape[1:])
    
    # get one row and col which are empty
    row_where_0, col_where_0, _ = np.where(mask == 0)
    random_idx = np.random.randint(row_where_0.shape[0])
    
    row = row_where_0[random_idx]
    col = col_where_0[random_idx]
    
    # activate/deactivate feature j
    mask[row, col] = 1
    mask_with_f = np.copy(mask)

    mask[row, col] = 0
    mask_without_f = np.copy(mask)

    # maksing data
    data_with_f = np.multiply(data, mask_with_f)
    data_without_f = np.multiply(data, mask_without_f)

    # get softmax value
    pred_with_f = np.max(model.predict(data_with_f), axis=1)
    pred_without_f = np.max(model.predict(data_without_f), axis=1)

    # marginal Shapely value
    diff = pred_with_f - pred_without_f
    diff = np.expand_dims(diff, axis=-1)
    importance[:, row, col] += diff
    count[:, row, col] += 1
    
    if i % 5000 == 0 and i > 0: # verbose
        print("For %dth iter, it takes %0.3f sec" % (i, time()-start_time))
        
        for data_num in range(0, data.shape[0]):
            img_name = "./images/fig_iter" + str(i) + "_data" + str(data_num) + ".png"
            pred = model.predict(np.expand_dims(data[data_num], axis=0))
                                 
            ax1 = plt.subplot(1, 2, 1)
            im1 = plt.imshow(np.squeeze(data[data_num]), cmap="gray")
            plt.title("Predicted as %d" % np.argmax(pred, axis=1))

            ax2 = plt.subplot(1, 2, 2)
            im2 = plt.imshow(np.squeeze(np.divide(importance[data_num], count[data_num])))
            plt.colorbar(im2, fraction=0.046, pad=0.04)
            plt.tight_layout()
            plt.savefig(img_name, dpi=300, bbox_inches="tight")
            plt.show()
        
        print("\n")
        start_time = time()
    
importance = np.divide(importance, count)